In [1]:
from policyengine.entities import create_db_and_tables
import h5py
import numpy as np
from sqlmodel import SQLModel, Field, create_engine, Session
from typing import Optional
import pandas as pd
from policyengine.entities import Entity, Variable
import time

engine = create_db_and_tables(connection_string="sqlite:///tax_policy_sqlite.db")

/opt/miniconda3/envs/py-3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Database created in 0.01 seconds


In [2]:
from policyengine_us import Microsimulation

def load_vars_to_sqlite(vars: dict):
    with Session(engine) as session:
        for var_name, var in vars.items():
            record = Variable(
                country_id="us",
                name=var_name,
                description=getattr(var, 'description', ''),
                dataset_series_id=1,
                data=None
            )

            session.add(record)

        session.commit()


# Load -us sim just to get variables in structured way
sim: Microsimulation = Microsimulation()

tbs = sim.tax_benefit_system
vars = tbs.variables

start_time = time.time()
load_vars_to_sqlite(vars)
end_time=time.time()
print(f"Variable metadata loaded in {end_time - start_time:.2f} seconds")

OperationalError: (sqlite3.OperationalError) table variable has no column named data
[SQL: INSERT INTO variable (created_at, country_id, name, description, data) VALUES (?, ?, ?, ?, ?) RETURNING id]
[parameters: ('2025-06-20 22:08:36.326148', 'us', 'employment_income_before_lsr', '', None)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:

def load_hdf5_to_sqlite(hdf5_path: str, dataset_name: str):
    with h5py.File(hdf5_path, 'r') as hdf5_file:
        dataset = hdf5_file[dataset_name]["2024"]

        with Session(engine) as session:
            for i, row in enumerate(dataset):
                record = Entity(
                    country_id="us",
                    entity_type="household",
                    dataset_id=1,  # Assuming a dataset ID of 1 for this example
                )

                session.add(record)
                
                # Commit in batches for better performance
                if i % 1000 == 0:
                    session.commit()
            
            session.commit()

start_time = time.time()
load_hdf5_to_sqlite("./enhanced_cps_2024.h5", "age")
end_time = time.time()
print(f"Households loaded in {end_time - start_time:.2f} seconds")


Households loaded in 13.14 seconds


In [ ]:
from policyengine import Simulation

pysim = Simulation(
    country_id="us",
    scope="macro",
    time_period=2025,
)

print(dir(pysim))

pysim.calculate_single_economy()

No data provided, using default dataset: gs://policyengine-us-data/cps_2023.h5
INFO:root:Using Google Cloud Storage for download.
INFO:root:Metadata for blob policyengine-us-data, cps_2023.h5 has version: 1.31.0
INFO:policyengine.utils.data.caching_google_storage_client:Syncing policyengine-us-data, cps_2023.h5, 1.31.0 to cache
INFO:root:Blob policyengine-us-data, /b/policyengine-us-data/o/cps_2023.h5 has version 1.31.0
INFO:policyengine.utils.data.simplified_google_storage_client:Crc for policyengine-us-data, cps_2023.h5 is mW2ZeQ==
INFO:root:Blob policyengine-us-data, /b/policyengine-us-data/o/cps_2023.h5 has version 1.31.0
INFO:policyengine.utils.data.simplified_google_storage_client:Downloaded policyengine-us-data, cps_2023.h5, version:1.31.0
INFO:policyengine.utils.data.caching_google_storage_client:Downloaded new version of policyengine-us-data, cps_2023.h5, 1.31.0 with crc mW2ZeQ==
INFO:policyengine.utils.data.caching_google_storage_client:Cache updated for {id_string}
INFO:poli

['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_output_functions', '_apply_region_to_simulation', '_initialise_simulation', '_initialise_simulations', '_set_data', '_set_data_from_gs', '_set_data_time_period', 'baseline_simulation', 'calculate_average_earnings', 'calculate_economy_comparison', 'calculate_household_comparison', 'calculate_single_economy', 'calculate_single_household', 'check_data_version', 'check_model_version', 'data_version', 'is_comparison', 'model_version', 'options', 'reform_simulation']
Variable 'household_count_people' written to database in 49.07 seconds
Variable 'household_count_people' written to database with strategy 'sql'
Var

AttributeError: 'NoneType' object has no attribute 'id'